In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

import torchvision.datasets as dset 

# **문제 1번**

**데이터 불러오기**

In [ ]:
transform = transforms.Compose([                 
     transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
trainset.shape()

AttributeError: ignored

**이미지확인**

imges.shape = [4,3,32,32]

images[0].shape = [3,32,32]

**conv2d + ReLU의 층을 더 쌓으면 이미지의 특성 처리하는 층이 늘어나 성능이 높아 질거라는 예상을 하고 시도1 모델에다가 conv2d+ ReLU층을 쌓아 시도2를 해보았고 시도2 모델에다가 conv2d + ReLU층을 쌓아 시도를 해보았다. 시도 1 ,2 ,3에서 output의 가로 픽셀수와 세로 픽셀수를 동일하게 해주고싶어 추가하는 conv2d층에다가는 padding = 2 인속성을 추가해주었다**

## **시도 1 : layers 14**

In [ ]:
# set parameters
learning_rate = 0.0004
num_epoch = 10


# CNN 모델학습
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels = 3,out_channels =16,kernel_size = 5),   # [4,3,32,32]->[16,3,28,28]
            nn.ReLU(),
            nn.MaxPool2d(2,2),                                             # [4,16,28,28]->[4,16,14,14]
            nn.Conv2d(in_channels = 16,out_channels =32,kernel_size = 5),  # [4,16,14,14]->[4,32,10,10]
            nn.ReLU(),
            nn.MaxPool2d(2,2),                                             # [4,32,10,10]->[4,32,5,5]
            nn.Conv2d(32,64,5,padding=2),                                  # [4,32,5,5]->[4,64,5,5]
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )

    def forward(self, x):
      out = self.layer(x)
      return out

net = Net()

# 손실함수와 옵티마이저 정하기
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
#신경망 학습하기
for epoch in range(2):   # 데이터셋을 수차례 반복합니다.
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs, labels = data

        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계를 출력합니다.
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.160
[1,  4000] loss: 1.751
[1,  6000] loss: 1.593
[1,  8000] loss: 1.508
[1, 10000] loss: 1.417
[1, 12000] loss: 1.365
[2,  2000] loss: 1.263
[2,  4000] loss: 1.240
[2,  6000] loss: 1.202
[2,  8000] loss: 1.188
[2, 10000] loss: 1.129
[2, 12000] loss: 1.132
Finished Training


In [ ]:
for data in testloader:
        images, labels = data
outputs = net(images)

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 61 %


## **시도 2: layers: 16**

In [ ]:
# set parameters
learning_rate = 0.0004
num_epoch = 10

# CNN 모델학습
import torch.nn as nn
import torch.nn.functional as F

class Net1(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels = 3,out_channels =16,kernel_size = 5),   # [4,3,32,32]->[16,3,28,28]
            nn.ReLU(),
            nn.MaxPool2d(2,2),                                             # [4,1628,28]->[4,16,14,14]
            nn.Conv2d(in_channels = 16,out_channels =32,kernel_size = 5),  # [4,16,14,14]->[4,32,10,10]
            nn.ReLU(),
            nn.MaxPool2d(2,2),                                             # [4,32,10,10]->[4,32,5,5]
            nn.Conv2d(32,64,5,padding=2),                                  # [4,32,5,5]->[4,64,5,5]
            nn.ReLU(),
            #--------------추가된레이어
            nn.Conv2d(64,192,5,padding=2),                                  # [4,64,5,5]->[4,192,5,5]
            nn.ReLU(),
            #---------------------
            nn.Flatten(),
            nn.Linear(192 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )

    def forward(self, x):
      out = self.layer(x)
      return out


net = Net1()

# 손실함수와 옵티마이저 정하기
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

path = "/content/data/model.pt"
torch.save(net, path)

In [ ]:
model = torch.load('model.pt')

#신경망 학습하기
for epoch in range(2): 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

[1,  2000] loss: 1.669
[1,  4000] loss: 1.564
[1,  6000] loss: 1.471
[1,  8000] loss: 1.407
[1, 10000] loss: 1.357
[1, 12000] loss: 1.329
[2,  2000] loss: 1.204
[2,  4000] loss: 1.180
[2,  6000] loss: 1.144
[2,  8000] loss: 1.138
[2, 10000] loss: 1.089
[2, 12000] loss: 1.064


In [ ]:
#정확도 체크
outputs = net(images)

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 63 %


## **시도 3: 18 layers**

In [ ]:
# set parameters
learning_rate = 0.0002
num_epoch = 10


# CNN 모델학습
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels = 3,out_channels =16,kernel_size = 5),   # [4,3,32,32]->[16,3,28,28]
            nn.ReLU(),
            nn.MaxPool2d(2,2),                                             # [4,16,28,28]->[4,16,14,14]
            nn.Conv2d(in_channels = 16,out_channels =32,kernel_size = 5),  # [4,16,14,14]->[4,32,10,10]
            nn.ReLU(),
            nn.MaxPool2d(2,2),                                             # [4,32,10,10]->[4,32,5,5]
            nn.Conv2d(32,64,5,padding=2),                                  # [4,32,5,5]->[4,64,5,5]
            nn.ReLU(),
            nn.Conv2d(64,192,5,padding=2),                                 # [4,64,5,5]->[4,192,5,5]
            nn.ReLU(),
            #--------------추가된레이어
            nn.Conv2d(192,256,5,padding=2),                                # [4,192,5,5]->[4,256,5,5]
            nn.ReLU(),
            nn.Conv2d(256,192,1),                                          #[4,256,5,5]->[4,192,5,5]
            nn.ReLU(),
            #---------------------
            nn.Flatten(),
            nn.Linear(192 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )

    def forward(self, x):
      out = self.layer(x)
      return out


net = Net()

# 손실함수와 옵티마이저 정하기
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
#신경망 학습하기
for epoch in range(2):  
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

[1,  2000] loss: 2.304
[1,  4000] loss: 2.303
[1,  6000] loss: 2.254
[1,  8000] loss: 2.019
[1, 10000] loss: 1.920
[1, 12000] loss: 1.871
[2,  2000] loss: 1.776
[2,  4000] loss: 1.661
[2,  6000] loss: 1.588
[2,  8000] loss: 1.550
[2, 10000] loss: 1.501
[2, 12000] loss: 1.453


In [ ]:
#신경망 학습하기
for data in testloader:
        images, labels = data
outputs = net(images)

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 50 %


**최종 성능 : 63%로 시도 2에서 가장 높은 성능을 기록했다**

# **문제 2번**

In [ ]:
# loading training data
from torchvision import datasets
import torchvision.transforms as transforms
import os

path2data = '/data'

# if not exists the path, make the path
if not os.path.exists(path2data):
    os.mkdir(path2data)

data_transformer = transforms.Compose([transforms.ToTensor()])
train_ds = datasets.STL10(path2data, split='train', download='True', transform=data_transformer)

print(train_ds.data.shape)

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting /data/stl10_binary.tar.gz to /data
(5000, 3, 96, 96)


In [ ]:
# load test dataset and call it test0_ds
test0_ds = datasets.STL10(path2data, split='test', download=True, transform=data_transformer)
print(test0_ds.data.shape)

Files already downloaded and verified
(8000, 3, 96, 96)


In [ ]:
batch_size = 4
trainloader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test0_ds, batch_size=batch_size, shuffle=True, num_workers=2)

 **이번에는 층을 늘리는 것이 아닌 처음 conv2d에서 내보내는 채널의 개수를 시도1에서는 16으로 시도 2에서는 32로 시도 3에서는 64로 늘릴 것이다 (이때 당연히 처음 conv2d의 아웃풋 채널수가 늘어나면 다음에 받는 conv2d의 채널수도 늘려준다 이미지와 내적하는 필터의 개수가 많아지는 것 이기때문에 아마도 개수가 늘어나면 더 좋은 성능을 낼 것이라고 예측한다**

In [ ]:
# set parameters
learning_rate = 0.0002
num_epoch = 10


# CNN 모델학습
import torch.nn as nn
import torch.nn.functional as F

class STLNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
             nn.Conv2d(3, 16, 5),                   #[4,3,96,96] -> [4,16,92,92]
             nn.ReLU(),
             nn.MaxPool2d(2, 2),                    #[4,16,92,92] -> [4,16,46,46]
             nn.Conv2d(16, 32, 5),                   #[4,16,46,46] -> [4,32,42,42]
             nn.ReLU(),
             nn.MaxPool2d(2, 2),                    #[4,32,42,42] -> [4,32,21,21]
             nn.ReLU(),
             nn.Flatten(), 
             nn.Linear(32 * 21 * 21, 120),     
             nn.ReLU(),       
             nn.Linear(120, 84),   
             nn.ReLU(),                 
             nn.Linear(84, 10),            
        )

    def forward(self, x):
      out = self.layer(x)
      return out


net = STLNet()

# 손실함수와 옵티마이저 정하기
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
#신경망 학습하기
for epoch in range(2):  
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

In [ ]:
#정확도 예측
for data in testloader:
        images, labels = data
outputs = net(images)

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 29 %


## **시도2 첫번째 conv의 out_channels을 16-> 32로 늘리고 두번째 conv2d의 out _chaneels를 32->64로 늘린다**

In [ ]:
# set parameters
learning_rate = 0.0002
num_epoch = 10


# CNN 모델학습
import torch.nn as nn
import torch.nn.functional as F

class STLNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
             nn.Conv2d(3, 32, 5),                   #[4,3,96,96] -> [4,32,92,92]
             nn.ReLU(),
             nn.MaxPool2d(2, 2),                    #[4,32,92,92] -> [4,32,46,46]
             nn.Conv2d(32, 64, 5),                   #[4,32,46,46] -> [4,64,42,42]
             nn.ReLU(),
             nn.MaxPool2d(2, 2),                    #[4,64,42,42] -> [4,64,21,21]
             nn.ReLU(),
             nn.Flatten(), 
             nn.Linear(64 * 21 * 21, 120),     
             nn.ReLU(),       
             nn.Linear(120, 84),   
             nn.ReLU(),                 
             nn.Linear(84, 10),            
        )

    def forward(self, x):
      out = self.layer(x)
      return out


net = STLNet()

# 손실함수와 옵티마이저 정하기
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
#신경망 학습하기
batch_size = 4
trainloader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test0_ds, batch_size=batch_size, shuffle=True, num_workers=2)

for epoch in range(2):  
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

In [ ]:
#신경망 학습하기
for data in testloader:
        images, labels = data
outputs = net(images)

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 32 %


## **시도 3 첫번째 conv2d의 out_channels를 32->64로 늘리고 두번째 conv2d의 out_channels를 64->128로 늘린다**

In [ ]:
# set parameters
learning_rate = 0.0002
num_epoch = 10


# CNN 모델학습
import torch.nn as nn
import torch.nn.functional as F

class STLNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
             nn.Conv2d(3, 64, 5),                   #[4,3,96,96] -> [4,32,92,92]
             nn.ReLU(),
             nn.MaxPool2d(2, 2),                    #[4,32,92,92] -> [4,32,46,46]
             nn.Conv2d(64, 128, 5),                  #[4,32,46,46] -> [4,64,42,42]
             nn.ReLU(),
             nn.MaxPool2d(2, 2),                    #[4,64,42,42] -> [4,64,21,21]
             nn.ReLU(),
             nn.Flatten(), 
             nn.Linear(128 * 21 * 21, 120),     
             nn.ReLU(),       
             nn.Linear(120, 84),   
             nn.ReLU(),                 
             nn.Linear(84, 10),            
        )

    def forward(self, x):
      out = self.layer(x)
      return out


net = STLNet()

# 손실함수와 옵티마이저 정하기
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
#신경망 학습하기
batch_size = 4
trainloader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test0_ds, batch_size=batch_size, shuffle=True, num_workers=2)

for epoch in range(2):  
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

In [ ]:
#신경망 학습하기
for data in testloader:
        images, labels = data
outputs = net(images)

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 34 %


**최종 성능: 34%  시도3에서 가장 높은 성능을 기록하였다**